In [1]:
import numpy as np 
import pandas as pd
from pandas import DataFrame,Series

data=pd.read_csv("F:\Python\data\Bankpep.csv",index_col="id")
print(data)

         age     sex      region    income married  children  car save_act  \
id                                                                           
ID12101   48  FEMALE  INNER_CITY  17546.00      NO         1   NO       NO   
ID12102   40    MALE        TOWN  30085.10     YES         3  YES       NO   
ID12103   51  FEMALE  INNER_CITY  16575.40     YES         0  YES      YES   
ID12104   23  FEMALE        TOWN  20375.40     YES         3   NO       NO   
ID12105   57  FEMALE       RURAL  50576.30     YES         0   NO      YES   
ID12106   57  FEMALE        TOWN  37869.60     YES         2   NO      YES   
ID12107   22    MALE       RURAL   8877.07      NO         0   NO       NO   
ID12108   58    MALE        TOWN  24946.60     YES         0  YES      YES   
ID12109   37  FEMALE    SUBURBAN  25304.30     YES         2  YES       NO   
ID12110   54    MALE        TOWN  24212.10     YES         2  YES      YES   
ID12111   66  FEMALE        TOWN  59803.90     YES         0   N

In [2]:
seq=["married","car","save_act","current_act","mortgage","pep"]
for feature in seq:#逐个特征进行替换
    data.loc[data[feature]=="YES",feature]=1
    data.loc[data[feature]=="NO",feature]=0
#替换性别
data.loc[data["sex"]=="FEMALE","sex"]=1
data.loc[data["sex"]=="MALE","sex"]=0
print(data[0:5])

         age sex      region   income married  children car save_act  \
id                                                                     
ID12101   48   1  INNER_CITY  17546.0       0         1   0        0   
ID12102   40   0        TOWN  30085.1       1         3   1        0   
ID12103   51   1  INNER_CITY  16575.4       1         0   1        1   
ID12104   23   1        TOWN  20375.4       1         3   0        0   
ID12105   57   1       RURAL  50576.3       1         0   0        1   

        current_act mortgage pep  
id                                
ID12101           0        0   1  
ID12102           1        1   0  
ID12103           1        0   0  
ID12104           1        0   0  
ID12105           0        0   0  


In [3]:
#将离散特征数据进行独立编码，转换为dummies矩阵
dumm_reg=pd.get_dummies(data["region"],prefix="region")
print(dumm_reg[0:5])

         region_INNER_CITY  region_RURAL  region_SUBURBAN  region_TOWN
id                                                                    
ID12101                  1             0                0            0
ID12102                  0             0                0            1
ID12103                  1             0                0            0
ID12104                  0             0                0            1
ID12105                  0             1                0            0


In [4]:
dumm_child=pd.get_dummies(data["children"],prefix="children")
print(dumm_child[0:2])

         children_0  children_1  children_2  children_3
id                                                     
ID12101           0           1           0           0
ID12102           0           0           0           1


In [5]:
#删除dataFrame中原有两列后再join dummies
df1=data.drop(["region","children"],axis=1)
# print(df1[0:5])
df2=df1.join([dumm_reg,dumm_child],how="outer")
print(df2[0:2])

         age sex   income married car save_act current_act mortgage pep  \
id                                                                        
ID12101   48   1  17546.0       0   0        0           0        0   1   
ID12102   40   0  30085.1       1   1        0           1        1   0   

         region_INNER_CITY  region_RURAL  region_SUBURBAN  region_TOWN  \
id                                                                       
ID12101                  1             0                0            0   
ID12102                  0             0                0            1   

         children_0  children_1  children_2  children_3  
id                                                       
ID12101           0           1           0           0  
ID12102           0           0           0           1  


In [6]:
#准备训练输入变量
X=df2.drop(["pep"],axis=1).values.astype(float)
y=df2["pep"].values.astype(int)
print("X.shape",X.shape)
print(X[0:2,:])
print(y[0:2])

X.shape (600, 16)
[[  4.80000000e+01   1.00000000e+00   1.75460000e+04   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.00000000e+01   0.00000000e+00   3.00851000e+04   1.00000000e+00
    1.00000000e+00   0.00000000e+00   1.00000000e+00   1.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[1 0]


In [7]:
#训练模型，评价分类器性能
from sklearn import svm
from sklearn import metrics
clf=svm.SVC(kernel="rbf",gamma=0.6,C=1.0)
clf.fit(X,y)
print("Accuracy:",clf.score(X,y))
y_predicted=clf.predict(X)
print(metrics.classification_report(y,y_predicted))

Accuracy: 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       326
          1       1.00      1.00      1.00       274

avg / total       1.00      1.00      1.00       600



In [8]:
#将数据集拆分为训练集和测试集，在测试集上查看分类效果
from sklearn import model_selection
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,test_size=0.3,random_state=1)
clf=svm.SVC(kernel="rbf",gamma=0.7,C=1.0)
clf.fit(X_train,y_train)
print("Performance on training set:",clf.score(X_train,y_train))
print("Performance on test set:",clf.score(X_test,y_test))

Performance on training set: 1.0
Performance on test set: 0.555555555556


In [9]:
#将标准化后的数据集拆分为训练集和测试集，在测试集上查看分类效果
from sklearn import preprocessing
X_scale=preprocessing.scale(X)
X_train,X_test,y_train,y_test=model_selection.train_test_split(X_scale,y,test_size=0.3,random_state=1)
clf=svm.SVC(kernel="poly",gamma=0.6,C=0.001)
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.805555555556


In [12]:
#查看在测试集上混淆矩阵，precision，recall，F1
y_predicted=clf.predict(X_test)
print("Classification report for %s"% clf)
print(metrics.classification_report(y_test,y_predicted))
print("Confusion matrix:\n",metrics.confusion_matrix(y_test,y_predicted))

Classification report for SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.6, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

          0       0.83      0.82      0.82       100
          1       0.78      0.79      0.78        80

avg / total       0.81      0.81      0.81       180

Confusion matrix:
 [[82 18]
 [17 63]]
